In [1]:
import os
import platform
import pandas as pd
import tensorflow as tf

from modules.dataset_utils import DatasetUtils as dataset
from tensorflow import keras
from tensorflow.keras import layers


#########################################################################################################################################################
# PARAMETROS DEFINIDOS EN FUNCION DEL USUARIO
#########################################################################################################################################################

# Ruta hacia los CSV del dataset
train_file_relative = './data/700Train.csv'
validation_file_relative = './data/700Validation.csv'
test_file_relative = './data/700Test.csv'

# Ruta al directorio de imágenes con máscara aplicada
masked_dir_relative = '../local/output_masked/'

# Ruta relativa hacia la librería ai4eutils
ai4eutils_relative = "./repos/ai4eutils"

# Ruta relativa hacia la librería CameraTraps
CameraTraps_relative = "./repos/CameraTraps"

#########################################################################################################################################################
# PARAMETROS PARA CNN
#########################################################################################################################################################

IMG_HEIGHT = IMG_WEIGHT = 700
BATCH_SIZE = 64

#########################################################################################################################################################
#########################################################################################################################################################

home = os.path.expanduser("~")

ai4utils = os.path.abspath(ai4eutils_relative)
CameraTraps = os.path.abspath(CameraTraps_relative)
train_file = os.path.abspath(train_file_relative)
validation_file = os.path.abspath(validation_file_relative)
test_file = os.path.abspath(test_file_relative)
masked_dir = os.path.abspath(masked_dir_relative)

try:
    os.environ['PYTHONPATH']
except KeyError:
    os.environ['PYTHONPATH'] = ""
if platform.system() == 'Windows':
    os.environ['PYTHONPATH'] += (";" + ai4utils)
    os.environ['PYTHONPATH'] += (";" + CameraTraps)
else:
    os.environ['PYTHONPATH'] += (":" + ai4utils)
    os.environ['PYTHONPATH'] += (":" + CameraTraps)

print('=======================================================================================================================================')
print('PYTHONPATH: ' + os.environ['PYTHONPATH'])
print('')
print('ai4eutils PATH: ' + '\t' + ai4utils)
print('CameraTraps PATH: ' + '\t' + CameraTraps)
print('Train CSV PATH: ' + '\t'  + train_file)
print('Validation CSV PATH: ' + '\t' + validation_file)
print('Test CSV PATH: ' + '\t' + '\t' + test_file)
print('masked PATH: ' + '\t' + '\t' + masked_dir)
print('=======================================================================================================================================')


TensorFlow version: 2.9.2
Is GPU available? tf.test.is_gpu_available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

PYTHONPATH: /Users/carlos/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles:/Users/carlos/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles/lib/python:/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/ai4eutils:/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/CameraTraps

ai4eutils PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/ai4eutils
CameraTraps PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/CameraTraps
Train CSV PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/data/700Train.csv
Validation CSV PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/data/700Validation.csv
Test CSV PATH: 		/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/data/700Test.csv
masked PATH: 		/Users/carlos/WORKSPACE/local/output_masked


In [ ]:
file_path, labels = dataset.load_csv(train_file)
file_path, labels = dataset.reset_path(file_path, labels)
file_path, labels = dataset.convert_to_abspath(masked_dir, file_path, labels)

file_path

In [ ]:
model = keras.Sequential([
    layers.Input((700, 700, 3)),
    layers.Conv2D(32, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    loss = [keras.losses.BinaryCrossentropy(from_logits=False)],
    optimizer = keras.optimizers.Adam(),
    metrics =  ['accuracy']
)
model.summary()

In [ ]:
model.fit(ds_train, epochs=10, batch_size=64)

In [ ]:
file_path, labels = dataset.load_csv(test_file)
file_path, labels = dataset.reset_path(file_path, labels)
file_path, labels = dataset.convert_to_abspath(masked_dir, file_path, labels)

ds_test = tf.data.Dataset.from_tensor_slices((file_path, labels))

ds_test = ds_test.map(dataset.load_image).map(dataset.resize_image)

print(ds_test)

In [ ]:
model.evaluate(ds_test)

In [ ]:
import random
import matplotlib.pyplot as plt

print(len(ds_test))
idx = random.randint(0, len(ds_test))
plt.imshow()
plt.show()